In [66]:
using PyCall
using TensorQEC
using Printf
using Base.Threads
stim = PyCall.pyimport("stim")
ldpc = PyCall.pyimport("ldpc")


noisy_circuit = stim.Circuit.from_file("bivariatebicyclecodes/r=6,d=6,p=0.001,noise=si1000,c=bivariate_bicycle_X,nkd=[[72,12,6]],q=144,iscolored=True,A_poly=x^3+y+y^2,B_poly=y^3+x+x^2.stim");

In [67]:
dem_flatten = noisy_circuit.detector_error_model(flatten_loops=true)
n_d, n_e = dem_flatten.num_detectors, dem_flatten.num_errors
error_rates = zeros(n_e)
stabilizer = zeros(Bool, n_d, n_e)
dem_str = split(dem_flatten.__str__(), "\n")
logical_ops_set = Set()
for i in 1:n_e
    s = split(dem_str[i])
    for j in 2:length(s)
        if startswith(s[j], "L")
            push!(logical_ops_set, parse(Int, s[j][2:end]))
        end
    end
end
logical_ops = zeros(Bool, length(logical_ops_set), n_e)
for i in 1:n_e
    s = split(dem_str[i])
    if startswith(s[1], "error(")
        num = parse(Float64, s[1][7:end-1])
        error_rates[i] = num
    end
    for j in 2:length(s)
        if startswith(s[j], "D")
            row = parse(Int, s[j][2:end])
            stabilizer[row+1, i] = 1
        end
        if startswith(s[j], "L")
            logical_ops[parse(Int, s[j][2:end])+1, i] = 1
        end
    end
end

In [68]:
n_d, n_e, size(logical_ops)

(432, 16200, (12, 16200))

In [75]:
num_samples = 1000
samples, logical_samples = noisy_circuit.compile_detector_sampler(
    seed=0).sample(num_samples, separate_observables=true);

In [77]:
bposd = ldpc.bposd_decoder(stabilizer, channel_probs=error_rates)
error_instances_bposd = []

t0 = time()
for i in 1:num_samples # segment default
    res = bposd.decode(samples[i, :])
    # res = bposd.osdw_decoding
    success = Bool.((logical_ops * res) .% 2) == logical_samples[i, :]
    if !success
        push!(error_instances_bposd, i)
        println(i, " ", logical_ops * res, " ", logical_samples[i, :])
    end

end
t1 = time()
@printf("bposd logical error rate: %.5f, time: %.2f\n", length(error_instances_bposd)/num_samples, t1-t0)

1 [0, 2, 2, 3, 2, 1, 1, 1, 1, 2, 0, 1] Bool[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1]
7 [0, 2, 2, 3, 0, 2, 0, 0, 1, 1, 2, 2] Bool[0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0]
9 [0, 2, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0] Bool[0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]
12 [0, 2, 2, 2, 0, 2, 0, 1, 1, 1, 2, 2] Bool[0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0]
14 [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1] Bool[1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1]
15 [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0] Bool[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
16 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] Bool[0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1]
18 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] Bool[0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]
21 [2, 3, 3, 3, 3, 0, 0, 2, 1, 2, 1, 2] Bool[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
24 [0, 3, 2, 1, 0, 2, 0, 0, 1, 0, 1, 1] Bool[0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1]
27 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] Bool[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
28 [0, 2, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0] Bool[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]
29 [0, 1, 1, 1, 1, 0, 0, 0, 0, 

In [78]:
decoder = IPDecoder()
res = decode(decoder, stabilizer, samples[1, :], error_rates)

TensorQEC.DecodingResult(true, Mod2[0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂  …  0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂, 0₂])

In [79]:
function extract_x(vec)
    return getfield.(vec, :x)
end
error_instances = []
decoder = IPDecoder()
t0 = time()
for i in 1:num_samples
    res = decode(decoder, stabilizer, samples[i, :], error_rates)
    success = Bool.((logical_ops * extract_x(res.error_qubits)) .% 2) == logical_samples[i, :]
    if !success
        push!(error_instances, i)
        println(i, " ", logical_ops * extract_x(res.error_qubits), ' ', logical_samples[i, :])
    end
end
t1 = time()
@printf("mip logical error rate: %.5f, time: %.2f\n", length(error_instances)/num_samples, t1-t0)

mip logical error rate: 0.00000, time: 2569.60


In [24]:
findall(!iszero, logical_ops[1, :])

1652-element Vector{Int64}:
    43
    44
    45
    46
   169
   170
   171
   172
   249
   250
     ⋮
 16123
 16143
 16151
 16153
 16173
 16174
 16175
 16179
 16194